# Create Daily Heists

In [1]:
import os, sys, random, json
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.utils import timezone
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import emporium.logic.guild as GD
import emporium.logic.stage as ST
import emporium.logic.simulation as SM
print([x for x in dir(EM) if x[0]!='_' and len(x)>8])

['AdvancedRoom', 'BasicRoom', 'CastleRoom', 'ExpeditionLevel', 'ExpeditionType', 'GothicTower', 'LeagueTrial', 'RequiredPower', 'RoomUpgrade', 'ThiefLevel', 'UniqueRoom', 'UnlockableItem', 'UnlockableThief']


In [7]:
import engine.models as GM
import engine.logic.content as CT
import engine.logic.launcher as LH
import engine.logic.resource as RS
import engine.logic.robot as RB
print([x for x in dir(GM) if x[0]!='_' and len(x)>8])

['GuildExpedition', 'GuildStage', 'ItemInGuild', 'ItemUnlocked', 'MarketStore', 'RoomInGuild', 'ThiefInGuild', 'ThiefUnlocked']


In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Development

In [9]:
# get user

userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [10]:
# view stages for all guilds

stageLs = GM.GuildStage.objects.filter().values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'id', 'BackgroundBias', 
                                     'ObstaclesL2', 'ObstaclesL3', 'ObstaclesL4', 'ObstaclesL5',], axis=1, errors='ignore')
print( len(stageLs) )

groupDf = stageDf[['GuildFK_id', 'Heist', 'StageNo']]
groupDf = groupDf.groupby(['GuildFK_id', 'Heist']).agg(['count']).reset_index()
groupDf.columns = [' '.join(str(c) for c in col) for col in groupDf.columns]
groupDf

24


,GuildFK_id,Heist,StageNo count
0,30,campaign,10
1,30,tower,12
2,30,trial,2


In [23]:
# get selected guild

guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)
PD.DataFrame([guildMd.__dict__]).drop(['_state', 'UserFK_id'], axis=1, errors='ignore')

,id,Name,LastPlayed,CreateDate,Selected,VaultGold,VaultStone,VaultGems,LastHeist,CampaignWorld,DungeonCheck
0,31,623,2025-01-13,2025-01-12,True,0,0,10,tower,1,2025-01-12


In [21]:
# reset stages

GM.GuildStage.objects.filter(GuildFK=guildMd).delete()
guildMd.DungeonCheck = None
guildMd.save()

stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd).values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
stageDf

(24, {'engine.GuildStage': 24})

""


In [13]:
# reset thieves

thiefLs = GM.ThiefInGuild.objects.filter(GuildFK=guildMd)
for th in thiefLs:
    th.Status = 'Ready'
    th.save()
print('thieves reset')

thieves reset


In [14]:
stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd).values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'id', 'GuildFK_id', 'BackgroundBias', 
                                     'ObstaclesL2', 'ObstaclesL3', 'ObstaclesL4', 'ObstaclesL5',], axis=1, errors='ignore')
len(stageLs)
stageDf#[12:15]

0

""


In [15]:
# test campaign world advancement

stageLs = GM.Guild Stage.objects.filter(GuildFK=guildMd, Heist='campaign')

for st in stageLs:

    actualLandings = [x for x in st.LandingTypes if x]
    numLandings = len(actualLandings)
    
    for ld in range(0, numLandings):
        if st.StageNo != len(stageLs) and ld != numLandings:
            st.LandingRewards[ld] = {'test': 'dummy'}
        if st.StageNo == len(stageLs) and ld < numLandings -1:
            st.LandingRewards[ld] = {'test': 'last'}

    if st.StageNo != len(stageLs):
        st.StageRewards = {'reward': 'test'}

    st.save()
    
stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd, Heist='campaign').values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'id', 'GuildFK_id', 'BackgroundBias', 'Background', 'Level', 
                                    'BaseRewards', 'Burgles', 'StageQueue', 'Assignments', 'Actions',
                                     'ObstaclesL1', 'ObstaclesL2', 'ObstaclesL3', 'ObstaclesL4', 'ObstaclesL5',], 
                                     axis=1, errors='ignore')
stageDf

SyntaxError: invalid syntax (834996697.py, line 3)